# Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import NoteComposeNet
from dataset import MidiDataset
from torch.utils.data import DataLoader
from train import TrainPipeline

import torch
import pandas as pd

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [3]:
BATCH_SIZE = 64
TRAIN_SAMPLES_PER_TRACK = 8
VALIDATE_SAMPLES_PER_TRACK = 2
EPOCHS = 8

In [4]:
model = NoteComposeNet()

In [5]:
CSV_PATH = r'datasets/midi-dataset.csv'
df = pd.read_csv(CSV_PATH)
train_midi = MidiDataset(df, context_len = model._context_len, samples_per_track=TRAIN_SAMPLES_PER_TRACK)
vali_midi = MidiDataset(df, context_len= model._context_len, samples_per_track=VALIDATE_SAMPLES_PER_TRACK)
del df

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=0.001, 
    weight_decay=0.004,
    betas=(0.9, 0.999),
    eps=1e-07, 
    amsgrad=False)
train_loader = DataLoader(
    train_midi, 
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=True, 
)
validation_loader = DataLoader(
    vali_midi, 
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=True
)

In [7]:
pipeline = TrainPipeline(train_loader, validation_loader, model, loss_fn, optimizer)
pipeline.train(EPOCHS)

  0%|          | 0/5 [00:03<?, ?batch/s]


KeyboardInterrupt: 